In [1]:
from ipywidgets import widgets, interact, interactive, fixed, interact_manual
from IPython.display import display

import warnings

import seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

warnings.filterwarnings('ignore')

In [2]:
precision = 3
values = [i*10**-precision for i in range(1,1001)]
e_max = 1.
e_values = [i*10**-precision for i in range(1,1000*int(e_max)+1)]
lambd = 0.0013581


# GUI components
n_slider = widgets.IntSlider(value=1000000, min=100, max=10000000, step=100, description='n')
a_slider = widgets.SelectionSlider(options=[("%g"%i,i) for i in values], value=0.010, description='alpha')
b_slider = widgets.SelectionSlider(options=[("%g"%i,i) for i in values], value=0.050, description='beta')
e_slider = widgets.SelectionSlider(options=[("%g"%i,i) for i in e_values], value=0.270, description='epsilon')
selection = widgets.RadioButtons(options=['n', 'alpha', 'beta', 'epsilon'],value='epsilon',description='Calculate:', disabled=False)

# Layout
slider_box = widgets.VBox([n_slider, a_slider, b_slider, e_slider])
box = widgets.HBox([slider_box, selection])

# FUNCTIONS

In [3]:
def calculateBeta(n, lambd):
    beta = 2*np.exp(-2*np.power(lambd,2)*n)
    return beta

In [4]:
%%latex
\begin{align}
 \beta={2e}^{-2{\lambda^2}n}
\end{align}

<IPython.core.display.Latex object>

In [5]:
def calculateLambdaFromNBeta(n, beta):
    lambd = np.sqrt(np.log(2/beta)/(2*n))
    return lambd

def calculateLambdaFromAlphaEpsilon(alpha, epsilon):
    lambd = alpha/(1+(np.exp(epsilon)))/(np.exp(epsilon)-1)
    return lambd

In [6]:
%%latex
\begin{align}
\lambda = \sqrt{\frac{\log{\frac{2}{\beta}}}{2n}}
\end{align}

<IPython.core.display.Latex object>

In [7]:
def calculateAlpha(n, beta, epsilon):
    lambd = calculateLambdaFromNBeta(n, beta)
    alpha = (1+(np.exp(epsilon)))/(np.exp(epsilon)-1)*lambd
    return alpha

In [8]:
%%latex
\begin{align}
\alpha = \frac{1+e^\epsilon}{e^\epsilon-1} \lambda =  \frac{1+e^\epsilon}{e^\epsilon-1} \sqrt{\frac{\log{\frac{2}{\beta}}}{2n}}
\end{align}

<IPython.core.display.Latex object>

In [9]:
def calculateEpsilon(n, alpha, beta):
    lambd = calculateLambdaFromNBeta(n, beta)
    epsilon = ((-alpha/lambd)-1)/(1-(alpha/lambd))
    return epsilon

In [10]:
%%latex
\begin{align}
\epsilon = \frac{\frac{-\alpha}{\lambda}-1}{1-\frac{\alpha}{\lambda}}
\end{align}

<IPython.core.display.Latex object>

In [11]:
def updateAlpha(n, beta, epsilon):
    a = calculateAlpha(n, beta, epsilon)
    a = np.round(a, precision)
    
    if(a < 0):
        print "Warning, a < 0"
        
    if(a > 1):
        print "Warning, a > 1"
        a = 1.
    
    return a
    
def updateBeta(n, alpha, epsilon):
    b = calculateBeta(n, lambd)
    b = np.round(b, precision)
    
    if(np.round(b, precision) <= 0):
        print "Warning, b < 0"
        b = 10**-precision
    if(b > 1):
        print "Warning, b > 1"
        b = 1.
    
    return b
     
def updateEpsilon(n, alpha, beta):
    e = calculateEpsilon(n, alpha, beta)
    e = np.round(e, precision)
    
    if(e < 0):
        print "Warning, e < 0"
        e = 10**-precision
    if(e > e_max):
        print "Warning, e > max"
        e = e_max
    
    return e

def update(n, alpha, beta, epsilon):
    print "Entering update..."
    print beta, alpha, epsilon
    
    locked = selection.get_interact_value()
    

    result = {
    'n': lambda n, alpha, beta, epsilon: n,
    'alpha': lambda n, alpha, beta, epsilon: updateAlpha(n, beta, epsilon),
    'beta': lambda n, alpha, beta, epsilon: updateBeta(n, alpha, epsilon),
    'epsilon': lambda n, alpha, beta, epsilon: updateEpsilon(n, alpha, beta) }
    
    sliders = {
        'n': n_slider,
        'alpha': a_slider,
        'beta': b_slider,
        'epsilon': e_slider
    }
    
    print result[locked](n, alpha, beta, epsilon)
    sliders[locked].value = np.round(result[locked](n, alpha, beta, epsilon), precision)


In [12]:
interact(update, n=n_slider, alpha=a_slider, beta=b_slider, epsilon=e_slider, continuous_update=False)
selection

interactive(children=(IntSlider(value=1000000, description=u'n', max=10000000, min=100, step=100), SelectionSl…

RadioButtons(description=u'Calculate:', index=3, options=('n', 'alpha', 'beta', 'epsilon'), value='epsilon')

# START HERE

In [13]:
display(box)